In [ ]:
!pip install pandas
!pip install sodapy

In [ ]:
from sodapy import Socrata
import pandas as pd
import time

In [ ]:
# Initialize Socrata client
client = Socrata("data.cityofnewyork.us", "YBLReD9Df7AMythxsFDNtXxZa", timeout=60)
dataset_id = "p5f6-bkga"
params = { # set the parameters and using where to set the limit
    "select": "created_date, borough, agency, agency_name, complaint_type, descriptor, location_type, incident_zip, city, status",
    "where": "complaint_type='Noise - Residential' AND created_date >= '2022-01-01T00:00:00'",
    "limit": 5000,
}
record_count = 100000

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from tenacity import retry, stop_after_attempt, wait_fixed

In [ ]:
@retry(stop=stop_after_attempt(5), wait=wait_fixed(5)) ## retry funtion to keep in loop and divide it by the chunk
def fetch_chunk(offset):
    return client.get(dataset_id, offset=offset, **params)

# Fetch data with retries
results, failed_offsets = [], []
for offset in range(0, record_count, params["limit"]):
    try:
        results.extend(fetch_chunk(offset))
    except Exception:
        failed_offsets.append(offset)

# Retry failed chunks
for offset in failed_offsets:
    try:
        results.extend(fetch_chunk(offset))
    except Exception:
        pass

In [ ]:
# Save results to CSV
pd.DataFrame.from_records(results).to_csv("noise_complaints.csv", index=False)